In [41]:
from sklearn.tree import DecisionTreeClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


clfs = {
    "CART": DecisionTreeClassifier(random_state=1234),
    "EBM": ExplainableBoostingClassifier(),
    "LR": LogisticRegression(random_state=1234),
    "GNB": GaussianNB()
    
}

In [42]:
# datasets = ['breast', 'campus', 'churn', 'climate',
#             'compas', 'diabetes', 'german', 'heart',
#             'stroke', 'student', 'thoracic', 'water']
#fix german dataset
#fix thoracic dataset

In [43]:
datasets = ['german']

In [44]:
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold
n_datasets = len(datasets)
n_splits = 5
n_repeats = 2
rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=1234)

auc_scores = np.zeros((len(clfs), n_datasets, n_splits * n_repeats))

In [45]:
from sklearn.base import clone 
from sklearn import metrics
import pandas as pd

import helper
import importlib
importlib.reload(helper)
from sklearn.pipeline import make_pipeline

for data_id, dataset in enumerate(datasets):
    X=pd.read_csv(f"datasets/cleaned/{dataset}_X.csv")
    X = X.drop("Unnamed: 0", axis=1)
    y = pd.read_csv(f"datasets/cleaned/{dataset}_y.csv")
    y = y.drop("Unnamed: 0", axis=1)

    features_types_df = pd.read_csv(f"datasets/cleaned/datatypes/{dataset}.csv")

    feature_inidices = list(map(int, list(features_types_df)))
    features_names = list(features_types_df.T[0])
    features_types = list(map(int, list(features_types_df.T[1])))

    preprocess = helper.select_preprocessing_for_many_feat(feature_inidices, features_types, features_names)

    for fold_id, (train, test) in enumerate(rskf.split(X, y)):
        for clf_id, clf_name in enumerate(clfs):
            clf = clone(clfs[clf_name])

            if(clf_name == "GNB" and dataset=="compas"):
                clf.fit(X.iloc[train], y.iloc[train])
                y_preds = clf.predict(X.iloc[test])
                fpr, tpr, thresholds = metrics.roc_curve(y.iloc[test], y_preds)
                auc_scores[clf_id, data_id, fold_id] = metrics.auc(fpr, tpr)
            else:
                clf_pipeline = make_pipeline(
                    preprocess,
                    clf
                )
                
                clf_pipeline.fit(X.iloc[train], y.iloc[train])
                y_preds = clf_pipeline.predict(X.iloc[test])
                fpr, tpr, thresholds = metrics.roc_curve(y.iloc[test], y_preds)
                auc_scores[clf_id, data_id, fold_id] = metrics.auc(fpr, tpr)

/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bart/anaconda3/envs/mainenv/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed w

In [46]:
auc_scores

array([[[0.54880952, 0.65833333, 0.65238095, 0.58452381, 0.58848921,
         0.64166667, 0.65357143, 0.65119048, 0.5452381 , 0.62769784]],

       [[0.64761905, 0.73809524, 0.68571429, 0.60595238, 0.63183453,
         0.66666667, 0.68690476, 0.63214286, 0.66904762, 0.6926259 ]],

       [[0.6452381 , 0.76190476, 0.66428571, 0.675     , 0.62218225,
         0.66428571, 0.65714286, 0.6702381 , 0.66309524, 0.71043165]],

       [[0.71547619, 0.61071429, 0.73095238, 0.61904762, 0.6513789 ,
         0.6       , 0.68809524, 0.72380952, 0.65952381, 0.69874101]]])